# Panda power three bus problem.

In [1]:
import pandapower as pp

# Create empty net
# https://pandapower.readthedocs.io/en/stable/elements/empty_network.html

net = pp.create_empty_network() 


In [2]:
# Create 3 buses, b1,b2,b3
# https://pandapower.readthedocs.io/en/stable/elements/bus.html
V_nom_kV = 220

b1 = pp.create_bus(net, vn_kv=V_nom_kV,  name="Bus 1")
b2 = pp.create_bus(net, vn_kv=V_nom_kV,  name="Bus 2")
b3 = pp.create_bus(net, vn_kv=V_nom_kV,  name="Bus 3")

net.bus

,name,vn_kv,type,zone,in_service,geo
0,Bus 1,220.0,b,None,True,None
1,Bus 2,220.0,b,None,True,None
2,Bus 3,220.0,b,None,True,None


In [3]:
###### create bus elements

# BUS 1 
# Create a swing bus, V=1.05 py Angle =0
# Note: Its called an external grid in PP
# https://pandapower.readthedocs.io/en/stable/elements/ext_grid.html

pp.create_ext_grid(net, bus=b1, vm_pu=1.05, name="Swing bus")


# BUS 2 
# Create a load bus, P=256.6MW, Q=110.2 Mvar
# https://pandapower.readthedocs.io/en/stable/elements/load.html
pp.create_load(net, bus=b2, p_mw=0.2566, q_mvar=0.1102, name="Load 2")


# BUS 3 
# Create a load bus, P=138.6MW, Q=45.2 Mvar
# https://pandapower.readthedocs.io/en/stable/elements/load.html
pp.create_load(net, bus=b3, p_mw=0.1386, q_mvar=0.0452, name="Load 3")

net.load


,name,bus,p_mw,q_mvar,const_z_p_percent,const_i_p_percent,const_z_q_percent,const_i_q_percent,sn_mva,scaling,in_service,type
0,Load 2,1,0.2566,0.1102,0.0,0.0,0.0,0.0,NaN,1.0,True,wye
1,Load 3,2,0.1386,0.0452,0.0,0.0,0.0,0.0,NaN,1.0,True,wye


In [4]:
## Create lines
#https://pandapower.readthedocs.io/en/stable/elements/line.html
# NOTE: if x,r in PU then set length_km=Z_base to get from PU values to actual values.

# Calculate Z_Base = V/I = V^2/VI
VA_nom_MVA = 100 
Z_base = (V_nom_kV)**2/VA_nom_MVA 
print ("Z_base = ",Z_base)

# Line 1-2 0.02+j0.04 

pp.create_line_from_parameters(net,from_bus=b1, to_bus=b2,length_km=Z_base, r_ohm_per_km = 0.02, x_ohm_per_km = 0.04, c_nf_per_km = 1e-9, max_i_ka=1000)

# Line 2-3 0.0125+j0.025 
pp.create_line_from_parameters(net,from_bus=b2, to_bus=b3,length_km=Z_base, r_ohm_per_km = 0.0125, x_ohm_per_km = 0.025, c_nf_per_km = 1e-9, max_i_ka=1000)

# Line 1-3 0.01+j0.03 

pp.create_line_from_parameters(net,from_bus=b1, to_bus=b3,length_km=Z_base, r_ohm_per_km = 0.01, x_ohm_per_km = 0.03, c_nf_per_km = 1e-9, max_i_ka=1000)

net.line

Z_base =  484.0


,name,std_type,from_bus,to_bus,length_km,r_ohm_per_km,x_ohm_per_km,c_nf_per_km,g_us_per_km,max_i_ka,df,parallel,type,in_service,geo
0,None,None,0,1,484.0,0.0200,0.040,1.000000e-09,0.0,1000.0,1.0,1,None,True,None
1,None,None,1,2,484.0,0.0125,0.025,1.000000e-09,0.0,1000.0,1.0,1,None,True,None
2,None,None,0,2,484.0,0.0100,0.030,1.000000e-09,0.0,1000.0,1.0,1,None,True,None


In [5]:
net.line

,name,std_type,from_bus,to_bus,length_km,r_ohm_per_km,x_ohm_per_km,c_nf_per_km,g_us_per_km,max_i_ka,df,parallel,type,in_service,geo
0,None,None,0,1,484.0,0.0200,0.040,1.000000e-09,0.0,1000.0,1.0,1,None,True,None
1,None,None,1,2,484.0,0.0125,0.025,1.000000e-09,0.0,1000.0,1.0,1,None,True,None
2,None,None,0,2,484.0,0.0100,0.030,1.000000e-09,0.0,1000.0,1.0,1,None,True,None


In [6]:
# Do a power flow
# https://pandapower.readthedocs.io/en/stable/powerflow/ac.html

pp.runpp(net, algorithm='gs');

In [7]:
# Save the network to a json file
# https://pandapower.readthedocs.io/en/stable/file_io.html

pp.to_json(net,"ThreeBus.json")

In [8]:
# View results on the busses
net.res_bus

,vm_pu,va_degree,p_mw,q_mvar
0,1.050000,0.000000,-0.395213,-0.15543
1,1.049938,-0.003283,0.256600,0.11020
2,1.049956,-0.002716,0.138600,0.04520


In [9]:
# View results on the lines
net.res_line

,p_from_mw,q_from_mvar,p_to_mw,q_to_mvar,pl_mw,ql_mvar,i_from_ka,i_to_ka,i_ka,vm_from_pu,va_from_degree,vm_to_pu,va_to_degree,loading_percent
0,0.191744,0.067674,-0.191737,-0.067659,7.500330e-06,0.000015,0.000508,0.000508,0.000508,1.050000,0.000000,1.049938,-0.003283,0.000051
1,-0.064863,-0.042541,0.064864,0.042542,6.822798e-07,0.000001,0.000194,0.000194,0.000194,1.049938,-0.003283,1.049956,-0.002716,0.000019
2,0.203469,0.087756,-0.203464,-0.087742,4.453561e-06,0.000013,0.000554,0.000554,0.000554,1.050000,0.000000,1.049956,-0.002716,0.000055
